In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_pandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

BUFFER_IN_METERS = 10 * 1000 # 10km
SNOW_EX_GEOJSON_FILENAME = "snow_ex_all_stations.geojson"
REFRESH_SNOW_EX_GEOJSON = False
SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

if REFRESH_SNOW_EX_GEOJSON or not os.path.isfile(SNOW_EX_GEOJSON_FILENAME):
    # Get the snow ex sites
    engine, session = get_db(SNOWEX_DB_NAME)
    qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.latitude, SiteData.longitude)

    df_snow_ex = query_to_pandas(qry, engine).drop_duplicates(subset=["site_id"])
    gdf_snow_ex = gpd.GeoDataFrame(
        df_snow_ex, 
        geometry=gpd.points_from_xy(df_snow_ex.longitude, df_snow_ex.latitude),
        crs=4326
    )
    gdf_snow_ex.to_file(SNOW_EX_GEOJSON_FILENAME, driver='GeoJSON')
    #gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)
    session.close()
else:
    gdf_snow_ex = gpd.read_file(SNOW_EX_GEOJSON_FILENAME)
    
print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snow Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex_buffer.to_crs(4326).explore()
snow_ex_map = gdf_snow_ex.to_crs(4326).explore(m=snow_ex_map)


# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
msnt_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=MSNT&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_msnt = gpd.read_file(msnt_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_sntl = pd.concat([gdf_msnt, gdf_sntl])
gdf_sntl = gdf_sntl[gdf_sntl["beginDate"].dt.year <= 2017]
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
).set_geometry("geometry")
gdf_sntl_clipped = gdf_sntl.clip(gdf_snow_ex_buffer)
sntl_map = gdf_sntl_clipped.explore()
snow_ex_map = gdf_sntl_clipped.explore(m=snow_ex_map)
print(f"{len(gdf_sntl_clipped)} AWDB sites were found within {BUFFER_IN_METERS / 1000:.0f} km of SnowEx Sites")
_ = [print(f'* {row["site_name"]} ({row["site_id"]})') for idx, row in gdf_sntl_clipped.sort_values("site_name").iterrows()]
valid_triplets = list(set(gdf_sntl_clipped["site_id"]))

The CRS of the Snow Ex metadata is - epsg:4326
The bounding box of the snow Ex sites is - [-120.29898   35.85793 -105.54616   44.30447]
The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:4326, all good!
28 AWDB sites were found within 10 km of SnowEx Sites
* Banner Summit (312:ID:SNTL)
* Bogus Basin (978:ID:SNTL)
* Brighton (366:UT:SNTL)
* Butte (380:CO:SNTL)
* Caples Lake (CAP:CA:MSNT)
* Carson Pass (1067:CA:SNTL)
* Devils Postpile (DPO:CA:MSNT)
* Fool Creek (1186:CO:SNTL)
* Forestdale Creek (1049:CA:SNTL)
* Idarado (538:CO:SNTL)
* Independence Camp (539:CA:SNTL)
* Independence Creek (540:CA:SNTL)
* Independence Lake (541:CA:SNTL)
* Joe Wright (551:CO:SNTL)
* Mammoth Pass (MHP:CA:MSNT)
* Mesa Lakes (622:CO:SNTL)
* Mill-D North (628:UT:SNTL)
* Mineral Creek (629:CO:SNTL)
* Mores Creek Summit (637:ID:SNTL)
* Niwot (663:CO:SNTL)
* Red Mountain Pass (713:CO:SNTL)
* Sawtooth (1251:CO:SNTL)
* Schneiders (SCN:CA:MSNT)
* Schofield Pass

In [ ]:
from datetime import datetime

import requests
import pandas as pd
import numpy as np

API_DOMAIN = "https://api.snowdata.info/"

def get_awdb_data(site_ids, element="WTEQ", sdate=datetime(1899,10,1), edate=datetime.now(), orient="records", server=API_DOMAIN, sesh=None):
    """
    Takes a list of site ids or a single site id and by default returns SWE period of record data as a single or list of dataframes,
    but user can pass args to modify data returned.
    
    Valid elements include WTEQ, SNWD, PREC, SMS, STO, TAVG
    site_id takes the form of a triplet made from <network_site_id>:<state_abbrv>:<network> where network is either SNTL or MNST
    """
    dfs = []
    return_single = False
    if not isinstance(site_ids, list):
        site_ids = [site_ids]
        return_single = True
    for site_id in site_ids:
        endpoint = "data/getDaily"
        date_args = f"sDate={sdate:%Y-%m-%d}&eDate={edate:%Y-%m-%d}"
        frmt_args = f"format=json&orient={orient}"
        all_args = f"?triplet={site_id}&{date_args}&element={element}&{frmt_args}"
        url = f"{server}{endpoint}{all_args}"
        print(f"getting data for {site_id} {element} starting {sdate:%Y-%m-%d} and ending {edate:%Y-%m-%d}")
        data_col_lbl = f"{site_id}" + ":" + f"{element}"
        if sesh:
            req = sesh.get(url)
        else:
            req = requests.get(url)
        if req.ok:
            df = pd.DataFrame.from_dict(req.json())
            df.columns = ["Date", data_col_lbl]
            df.set_index("Date", inplace=True)
        else:
            print("  No data returned!")
            df = pd.DataFrame(
                data=[
                    {"Date": pd.NaT, data_col_lbl: np.nan}
                ],
            ).set_index("Date").dropna()
        dfs.append(df)
    if return_single:
        return dfs[0]
    return dfs

with requests.Session() as sesh:
    dfs_swe = get_awdb_data(site_ids=valid_triplets, element="WTEQ", sdate=datetime(2016, 10, 1), sesh=sesh)
    dfs_snwd = get_awdb_data(site_ids=valid_triplets, element="SNWD", sdate=datetime(2016, 10, 1), sesh=sesh)
df_all_swe_data = pd.concat(dfs_swe, axis=1)
df_all_snwd_data = pd.concat(dfs_snwd, axis=1)
df_all_awdb_data = pd.concat([df_all_swe_data, df_all_snwd_data], axis=1).fillna(np.nan)
df_all_awdb_data.index = pd.to_datetime(df_all_awdb_data.index)
df_all_awdb_data.tail(2)


getting data for 978:ID:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for DPO:CA:MSNT WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 540:CA:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 539:CA:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 814:UT:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14
getting data for 366:UT:SNTL WTEQ starting 2016-10-01 and ending 2022-07-14


In [ ]:
engine, session = get_db(SNOWEX_DB_NAME)
use_cols = ("site_id", "date", "pit_id", "value", "geometry", "depth", "instrument")
obj = {}
data_dfs = []
for awdb_site_id in valid_triplets[4:]:
    gdf_awdb_site_buffer = gdf_sntl[gdf_sntl["site_id"] == awdb_site_id].to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
    gdf_snow_ex_inside = gdf_snow_ex.clip(gdf_awdb_site_buffer)
    snow_ex_site_list = gdf_snow_ex_inside["site_id"].tolist()
    awdb_site_name = gdf_sntl[gdf_sntl["site_id"] == awdb_site_id]["site_name"].values[0]
    print(
        f"The AWDB site: {awdb_site_name} has {len(snow_ex_site_list)} SnowEx pits within "
        f"{BUFFER_IN_METERS / 1000:.0f} km ({', '.join(snow_ex_site_list)})."
    )
    
    qry = session.query(LayerData).filter(LayerData.site_id.in_(snow_ex_site_list)).filter(LayerData.type == "density")
    df_snow_ex_data = query_to_pandas(qry, engine)
    gdf_snow_ex_data = gpd.GeoDataFrame(
        df_snow_ex_data, 
        geometry=gpd.points_from_xy(df_snow_ex_data.longitude, df_snow_ex_data.latitude),
        crs=4326
    )
    gdf_snow_ex_data.drop(columns=[i for i in df_snow_ex_data.columns if i not in use_cols], inplace=True)
    gdf_snow_ex_data.loc[:, "value"] = gdf_snow_ex_data["value"].astype(float)
    gdf_snow_ex_data.index = pd.to_datetime(gdf_snow_ex_data["date"])
    df_grp_by = gdf_snow_ex_data.groupby(by="pit_id")
    df_date = df_grp_by["date"].first()
    df_bulk_density = df_grp_by["value"].mean() / 1000 # convert kg/m^3
    df_depth = df_grp_by["depth"].max() * 10 # convert cm to mm
    df_swe = df_depth * df_bulk_density
    df_pit_id = df_grp_by["pit_id"].first()
    df_values = gpd.GeoDataFrame(data={"swe_snowex": df_swe.round(0), "depth_snowex": df_depth, "date": df_date})#.set_geometry("geometry")
    df_values.index = df_pit_id.values#pd.to_datetime(df_values["date"])
    gdf_snow_ex_data.drop(columns="date", inplace=True)
    df_pit_data = df_grp_by.first().drop(columns=["depth", "value"])
    df_combined = df_pit_data.join(df_values).rename(columns={"site_id": "snowex_id", "pit_id": "snowex_pit_id"})
    df_awdb_site = df_all_awdb_data[[f"{awdb_site_id}:WTEQ", f"{awdb_site_id}:SNWD"]]
    df_combined = df_combined.reset_index()
    df_combined.index = df_date
    df_combined = df_combined.join((df_awdb_site / 0.0393701).round(0)).rename(columns={f"{awdb_site_id}:WTEQ": "swe_awdb", f"{awdb_site_id}:SNWD": "depth_awdb"})
    
    df_combined["awdb_id"] = awdb_site_id
    df_combined.set_crs(4326, inplace=True)
    sntl_point = gdf_sntl_clipped[gdf_sntl_clipped["site_id"] == awdb_site_id]["geometry"].to_crs(26912).values[0]
    df_combined["distance"] = df_combined.to_crs(26912).distance(sntl_point).round(0)
    data_dfs.append(df_combined)

obj["data"] = pd.concat(data_dfs).to_crs(4326)
obj["meta"] = {
    "awdb": gdf_sntl_clipped.to_crs(4326),
    "snowex": gdf_snow_ex.to_crs(4326)
}

In [ ]:
import pickle
import json

with open("one_obj_to_rule_them_all.pkl", "wb") as p:
    pickle.dump(obj, p)

with open("one_obj_to_rule_them_all.json", "w") as j:
    json.dump(
        {
            "data": json.loads(pd.concat(data_dfs).to_crs(4326).astype({"date": str}).to_json()),
            "meta": {
                "awdb": json.loads(gdf_sntl_clipped.to_crs(4326).to_json()),
                "snowex": json.loads(gdf_snow_ex.to_crs(4326).to_json())
            }
        },
        j,
        indent=4
    )

In [ ]:
obj["data"].sort_values("distance").head(10)

In [ ]:
print(
    "Here's a list of SNOTEL stations from nearest to furthest for use in the next cell",
    ", ".join([i for i in obj["data"].sort_values("distance")["awdb_id"].unique()])
)

In [ ]:
import folium
from folium.plugins import HeatMap

heat_map_sntl = "766:UT:SNTL"
gdf = obj["data"].sort_values("distance")
gdf = gdf[gdf["awdb_id"] == heat_map_sntl].astype({"date": str})
gdf["SnowEx - Snotel Delta SWE (in)"] = gdf["swe_snowex"] - gdf["swe_awdb"]
m_heat_map = gdf.explore(
    column="SnowEx - Snotel Delta SWE (in)",
    scheme="naturalbreaks",
    legend=True,
    k=5,
    legend_kwds=dict(colorbar=False),
    name="Snow Pits",
)
sntl_meta = obj["meta"]["awdb"][obj["meta"]["awdb"]["site_id"] == heat_map_sntl]
sntl_geo = sntl_meta["geometry"].values[0]
sntl_name = sntl_meta["site_name"].values[0]
folium.Marker(
    [sntl_geo.y, sntl_geo.x],
    popup=f"<b>{sntl_name}</b> ({heat_map_sntl})",
    tooltip=f"{sntl_name}"
).add_to(m_heat_map)
heat_map_data = []
for idx, row in gdf.iterrows():
    heat_pnt = [row["geometry"].y, row["geometry"].x, row["SnowEx - Snotel Delta SWE (in)"]]
    heat_map_data.append(heat_pnt)
heat_map_lyr = HeatMap(
    heat_map_data, 
    name="Heat Map", 
    min_opacity=0.5,
    max_zoom=18, 
    radius=25, 
    blur=15, 
    gradient=None, 
    overlay=True, 
    control=True, 
    show=True
).add_to(m_heat_map)
folium.map.LayerControl().add_to(m_heat_map)
m_heat_map


In [ ]:
import plotly.express as px

fig = px.box(gdf, y="SnowEx - Snotel Delta SWE (in)")
fig